# Splom plots

Splom plots are scatterplot matrices in which each cell represents a pairwise scatterplot of the input sequences. A collection of n sequences results in n*n scatterplots of which the diagonal cells are bisectors through the origin

In [ ]:
// Referencing Plotly.NET.Interactive extension
#r "nuget: Plotly.NET.Interactive, 3.0.2"
// Deedle is required for data handling
#r "nuget: Deedle.Interactive, 3.0.0-beta.1"

open Plotly.NET
open Plotly.NET.StyleParam
open Plotly.NET.LayoutObjects
open Plotly.NET.TraceObjects
open Deedle

Installed Packages Deedle.Interactive, 3.0.0-beta.1 Plotly.NET.Interactive, 3.0.2

In [ ]:
// read csv as data frame
let data = Frame.ReadCsv(__SOURCE_DIRECTORY__ + @"\data\iris-data.csv")

data

0,->,5.1,3.5,1.4,0.2,Iris-setosa
1,->,4.9,3.0,1.4,0.2,Iris-setosa
2,->,4.7,3.2,1.3,0.2,Iris-setosa
3,->,4.6,3.1,1.5,0.2,Iris-setosa
4,->,5.0,3.6,1.4,0.2,Iris-setosa
:,,...,...,...,...,...
145,->,6.7,3.0,5.2,2.3,Iris-virginica
146,->,6.3,2.5,5.0,1.9,Iris-virginica
147,->,6.5,3.0,5.2,2.0,Iris-virginica
148,->,6.2,3.4,5.4,2.3,Iris-virginica
149,->,5.9,3.0,5.1,1.8,Iris-virginica


In [ ]:
// extract values as float sequences
let sepalLengthData = data.["sepal length"] |> Series.values
let sepalWidthData  = data.["sepal width"]  |> Series.values
let petalLengthData = data.["petal length"] |> Series.values
let petalWidthData  = data.["petal width"]  |> Series.values

// a color scale value is defined for each species
let colors = 
    data.GetColumn<string> "class" 
    |> Series.values
    |> Seq.map (fun x ->
        match x with
        | "Iris-setosa" -> 0.
        | "Iris-versicolor" -> 0.5
        | _ -> 1.
        )
    |> Color.fromColorScaleValues

In [ ]:
let splomChart = 
    Chart.Splom(
        // input is a sequence of <identifier*values>
        [
            "sepal length" , sepalLengthData
            "sepal width"  , sepalWidthData
            "petal length" , petalLengthData
            "petal width"  , petalWidthData
        ],
        MarkerColor = colors,
        MarkerColorScale=StyleParam.Colorscale.Portland
    )
    |> Chart.withLayout(
        Layout.init(
            // hover info is shown just for closest data point
            HoverMode = StyleParam.HoverMode.Closest,
            // if a region is selected, data points within the selection are highlighted
            DragMode = StyleParam.DragMode.Select
        )
    )
    |> Chart.withSize (1000,1000)
    // alters the "Download plot" button to export the plot as SVG 
    |> Chart.withConfig(Config.init(ToImageButtonOptions = ConfigObjects.ToImageButtonOptions.init(Format=ImageFormat.SVG)))
    //|> Chart.withTitle(Title.init("Splom plot of iris data set",X=0.5))
let setosaAnnotation = Annotation.init(Text="Iris setosa",X=7.6,Y=7.6,BGColor= Color.fromHex "#D99090",BorderColor= Color.fromString "black", BorderPad=4)
let versicolorAnnotation = Annotation.init(Text="Iris versicolor",X=5.4,Y=5.4,BGColor= Color.fromHex "#F2E49F",BorderColor= Color.fromString "black", BorderPad=4)
let virginicaAnnotation = Annotation.init(Text="Iris virginica",X=4.7,Y=4.7,BGColor= Color.fromHex "#7798DB",BorderColor= Color.fromString "black", BorderPad=4)

splomChart
|> Chart.withAnnotations [setosaAnnotation;versicolorAnnotation;virginicaAnnotation]

<!-- Plotly chart will be drawn inside this DIV -->